In [33]:
import xtgeo
import resqpy
import numpy as np
import pandas as pd
from resqpy.model import Model, new_model
from resqpy.surface import Mesh
from resqpy.crs import Crs

from fmu.sumo.explorer import Explorer
sumo = Explorer("dev")
surface = sumo.get_surface_by_uuid("87f9423f-4d9d-9ef1-fb41-e6be452f0a17")

In [34]:
surfdata = surface.metadata['data']
surfdata

{'tagname': 'DS_extract_geogrid',
 'is_prediction': True,
 'bbox': {'ymin': 5926551.219238281,
  'zmin': 1600.4984130859375,
  'xmin': 456063.69140625,
  'ymax': 5939431.219238281,
  'zmax': 2000.4561767578125,
  'xmax': 467483.69140625},
 'format': 'irap_binary',
 'name': 'VOLANTIS GP. Base',
 'stratigraphic': True,
 'vertical_domain': 'depth',
 'is_observation': False,
 'content': 'depth',
 'spec': {'ncol': 572,
  'yori': 5926551.219238281,
  'xori': 456063.69140625,
  'rotation': 0,
  'undef': 1e+30,
  'yinc': 20,
  'xinc': 20,
  'nrow': 645,
  'yflip': 1}}

Instantiate new model of surface 

In [35]:
#model = Model(new_epc=True)
model = new_model(epc_file="surface.epc")

Add coordinate reference system of the surface metadata to the model

In [36]:
x_offset = surfdata['spec']['xori']
y_offset = surfdata['spec']['yori']
rotation = surfdata['spec']['rotation']
z_inc_down = surfdata['spec']['yflip'] # Determines "handedness" of coordinate system https://en.wikipedia.org/wiki/Right-hand_rule#Coordinates

crs = Crs(model, x_offset=x_offset, y_offset=y_offset, rotation=rotation, title="Surface Coordinate Reference System", z_inc_down=bool(z_inc_down))

Add a mesh (2d grid) of the surface data to the model

In [37]:
surf = xtgeo.surface_from_file(surface.blob)

# Here xy(z) -> ij where i = x, j = y dirs
origin = (0,0,0)
ni = surfdata['spec']['nrow']
nj = surfdata['spec']['ncol']
dxyz_dij = np.array([[surfdata['spec']['xinc'], 0, 0],
                     [0, surfdata['spec']['yinc'], 0]])
z_values = np.where(surf.values.mask, np.nan, surf.values.data) # Turn masked array into normal array with values masked out as nan

mesh = Mesh(model, z_values=z_values, origin=origin, ni=ni, nj=nj, dxyz_dij=dxyz_dij, crs_uuid = crs.uuid, title="Surface")

# Append whole fmu metadata dict to mesh
mesh.append_extra_metadata(surface.metadata)

Output our current surface model

In [38]:
# Write all metadata to epc file
crs.create_xml()
mesh.create_xml()
model.store_epc("surface.epc")

# Write data to hdf5 file
mesh.write_hdf5()
model.create_hdf5_ext(file_name="surface.h5")

<Element {http://www.energistics.org/energyml/data/commonv2}EpcExternalPartReference at 0x28ede8b9080>

Try to reopen a new model from the files we just saved

In [39]:
# Recreate the model from epc file
read_model = Model("surface.epc")
read_model.parts()

['obj_LocalDepth3dCrs_645e5128-197b-11ee-bfdd-00d49ee52a08.xml',
 'obj_Grid2dRepresentation_64d3022d-197b-11ee-9f1e-00d49ee52a08.xml']

In [40]:
read_model.titles()

['Surface Coordinate Reference System', 'Surface']

In [41]:
read_model.h5_file_name()

'surface.h5'

In [42]:
read_model.h5_uuid()

UUID('645cac0f-197b-11ee-b626-00d49ee52a08')

Check that stored array is intact

In [43]:
read_mesh_uuid = read_model.uuid(obj_type="obj_Grid2dRepresentation")

read_mesh = Mesh(read_model, uuid=read_mesh_uuid)
print("Shape:", read_mesh.full_array_ref().shape)
print("Sample array piece:", read_mesh.full_array_ref()[206, 41])
print("Sample array piece:", read_mesh.full_array_ref()[206, 206])

Shape: (572, 645, 3)
Sample array piece: [ 820. 4120.   nan]
Sample array piece: [4120.         4120.         1724.98217773]


In [44]:
print("Shape:", mesh.full_array_ref().shape)
print("Sample array piece:", mesh.full_array_ref()[206, 41])
print("Sample array piece:", read_mesh.full_array_ref()[206, 206])

Shape: (572, 645, 3)
Sample array piece: [ 820. 4120.   nan]
Sample array piece: [4120.         4120.         1724.98217773]


Check that extra metadata is intact

In [45]:
read_mesh.extra_metadata

{'masterdata': "{'smda': {'field': [{'identifier': 'DROGON', 'uuid': '00000000-0000-0000-0000-000000000000'}]}}",
 'file': "{'checksum_md5': '5818fdd860b445b41c337fd9d8150384', 'relative_path': 'realization-0/iter-0/share/results/maps/basevolantis--ds_extract_geogrid.gri'}",
 'access': "{'asset': {'name': 'Drogon'}}",
 'data': "{'tagname': 'DS_extract_geogrid', 'is_prediction': True, 'bbox': {'ymin': 5926551.219238281, 'zmin': 1600.4984130859375, 'xmin': 456063.69140625, 'ymax': 5939431.219238281, 'zmax': 2000.4561767578125, 'xmax': 467483.69140625}, 'format': 'irap_binary', 'name': 'VOLANTIS GP. Base', 'stratigraphic': True, 'vertical_domain': 'depth', 'is_observation': False, 'content': 'depth', 'spec': {'ncol': 572, 'yori': 5926551.219238281, 'xori': 456063.69140625, 'rotation': 0, 'undef': 1e+30, 'yinc': 20, 'xinc': 20, 'nrow': 645, 'yflip': 1}}",
 'fmu': "{'context': {'stage': 'realization'}, 'iteration': {'name': 'iter-0'}, 'case': {'name': 'rowh_kmd_bleed_fri01', 'user': {'id': 

In [46]:
mesh.extra_metadata

{'masterdata': "{'smda': {'field': [{'identifier': 'DROGON', 'uuid': '00000000-0000-0000-0000-000000000000'}]}}",
 'file': "{'checksum_md5': '5818fdd860b445b41c337fd9d8150384', 'relative_path': 'realization-0/iter-0/share/results/maps/basevolantis--ds_extract_geogrid.gri'}",
 'access': "{'asset': {'name': 'Drogon'}}",
 'data': "{'tagname': 'DS_extract_geogrid', 'is_prediction': True, 'bbox': {'ymin': 5926551.219238281, 'zmin': 1600.4984130859375, 'xmin': 456063.69140625, 'ymax': 5939431.219238281, 'zmax': 2000.4561767578125, 'xmax': 467483.69140625}, 'format': 'irap_binary', 'name': 'VOLANTIS GP. Base', 'stratigraphic': True, 'vertical_domain': 'depth', 'is_observation': False, 'content': 'depth', 'spec': {'ncol': 572, 'yori': 5926551.219238281, 'xori': 456063.69140625, 'rotation': 0, 'undef': 1e+30, 'yinc': 20, 'xinc': 20, 'nrow': 645, 'yflip': 1}}",
 'fmu': "{'context': {'stage': 'realization'}, 'iteration': {'name': 'iter-0'}, 'case': {'name': 'rowh_kmd_bleed_fri01', 'user': {'id': 

Check array data intact using resqpy built in method

In [47]:
read_mesh.surface().sample_z_at_xy_points(np.array([[200, 200], [200, 205]]))

array([nan, nan])

In [48]:
read_mesh.surface().sample_z_at_xy_points(np.array([[4000, 4000], [4000, 4001]]))

array([1728.10559082, 1728.11261597])